# <div align="center" style="color: #ff5733;">Data Requested By Annastasia for Monthly Reporting</div>

In [1]:
# %% [markdown]
# # Jupyter Notebook Loading Header
#
# This is a custom loading header for Jupyter Notebooks in Visual Studio Code.
# It includes common imports and settings to get you started quickly.

# %% [markdown]
## Import Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from google.cloud import bigquery
import os
path = r'C:\Users\DwaipayanChakroborti\AppData\Roaming\gcloud\legacy_credentials\dchakroborti@tonikbank.com\adc.json'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = path
client = bigquery.Client(project='prj-prod-dataplatform')

# %% [markdown]
## Configure Settings
# Set options or configurations as needed
# Example: pd.set_option('display.max_columns', None)

In [2]:
# Year and Month for which data is extracted
data_year = 2024
data_month = 12

In [3]:
# How many Applications applied to each month

sq = """
# loan applied
SELECT 
    DATE_TRUNC( startApplyDateTime,MONTH) as mm, 
    case when reloan_flag = 1  and loantype not like 'FLEXUP' then 'Reloan'
         when loantype = 'FLEXUP' and new_loan_type = 'Flex-up' and reloan_flag = 0 and flagDisbursement = 1 then 'Flex-up' 
         
         else new_loan_type end as LoanProduct,
    count (distinct digitalLoanAccountId) as StartedApps
FROM 
    `prj-prod-dataplatform.risk_credit_mis.loan_master_table` 

group by 1,2
order by 1 desc,2;
"""

StartedApps = client.query(sq).to_dataframe(progress_bar_type='tqdm')
print(f"The rows and columns in StartedApps are:\t {StartedApps.shape}")

Job ID 228ca48b-572d-46a8-aa9c-433a31f32388 successfully executed: 100%|██████████|
Downloading: 100%|██████████|
The rows and columns in StartedApps are:	 (235, 3)


In [4]:
# How many submitted applications to each month
sq = """
# loan Submitted
SELECT 
    DATE_TRUNC((case when loantype = 'FLEXUP' and new_loan_type = 'Flex-up' and reloan_flag = 0 and flagDisbursement = 1 then startApplyDateTime else termsAndConditionsSubmitDateTime end),MONTH) as mm,  
    case when reloan_flag = 1 and loantype not like 'FLEXUP'then 'Reloan'
         when loantype = 'FLEXUP' and new_loan_type = 'Flex-up' and reloan_flag = 0 and flagDisbursement = 1 then 'Flex-up' 
                  else new_loan_type end as LoanProduct,
    count (distinct digitalLoanAccountId) as SubmittedApps
FROM 
    `prj-prod-dataplatform.risk_credit_mis.loan_master_table` 
group by 1,2
order by 1 desc,2;
"""

SubmittedApps = client.query(sq).to_dataframe(progress_bar_type='tqdm')
print(f"The rows and columns of SubmittedApps are:\t {SubmittedApps.shape}")
SubmittedApps.sample(10)

Job ID ae9e0fc3-7517-4694-9b68-c59eb9cd12e3 successfully executed: 100%|██████████|
Downloading: 100%|██████████|
The rows and columns of SubmittedApps are:	 (233, 3)


,mm,LoanProduct,SubmittedApps
106,2023-09-01,SIL-Instore,5598
201,2022-02-01,SIL-Instore,1
98,2023-10-01,Quick,20685
18,2024-11-01,Quick,45564
196,2022-03-01,Quick,14413
198,2022-03-01,SIL-Online(discontinued),394
104,2023-09-01,Quick,21791
211,2021-11-01,SIL-Online(discontinued),551
72,2024-02-01,Flex,113
208,2021-12-01,SIL-Online(discontinued),215


In [5]:
# Approved Loans for each month

sq = """
# Approved loans
SELECT 
DATE_TRUNC((case when loantype = 'FLEXUP' and new_loan_type = 'Flex-up' and reloan_flag = 0 and flagDisbursement = 1 then startApplyDateTime 
                  when reloan_flag = 1 and loantype not like 'FLEXUP' then startApplyDateTime else decision_date end),MONTH) as mm,
        case when reloan_flag = 1 and loantype not like 'FLEXUP'then 'Reloan'
         when loantype = 'FLEXUP' and new_loan_type = 'Flex-up' and reloan_flag = 0 and flagDisbursement = 1 then 'Flex-up' 
                  else new_loan_type end as LoanProduct,
    count (distinct digitalLoanAccountId) as ApprovedApps
FROM 
    `prj-prod-dataplatform.risk_credit_mis.loan_master_table`
where 
(case when loantype = 'FLEXUP' and new_loan_type = 'Flex-up' and reloan_flag = 0 and flagDisbursement = 1 then flagDisbursement 
      when reloan_flag = 1 and loantype not like 'FLEXUP' then flagDisbursement  else flagApproval end) = 1
group by 1,2
order by 1 desc,2
;
"""

ApprovedApps = client.query(sq).to_dataframe(progress_bar_type='tqdm')
print(f"The rows and columns in ApprovedApps are:\t {ApprovedApps.shape}")
ApprovedApps.head(10)

Job ID f7db8dfb-9029-44ed-abc0-1e160fb7e57f successfully executed: 100%|██████████|
Downloading: 100%|██████████|
The rows and columns in ApprovedApps are:	 (194, 3)


,mm,LoanProduct,ApprovedApps
0,2025-01-01,ACL TSA,9
1,2025-01-01,Flex-up,34
2,2025-01-01,Quick,167
3,2025-01-01,Reloan,26
4,2025-01-01,SIL Competitor,40
5,2025-01-01,SIL Repeat,38
6,2025-01-01,SIL ZERO,44
7,2025-01-01,SIL-Instore,753
8,2024-12-01,ACL TSA,735
9,2024-12-01,Flex-up,724


In [6]:
# Booked Loans for each month

sq = """
#Booked loans
SELECT 
DATE_TRUNC( disbursementDateTime,MONTH) as mm, 
            case when reloan_flag = 1 and loantype not like 'FLEXUP' and flagDisbursement = 1 then 'Reloan'
         when loantype = 'FLEXUP' and new_loan_type = 'Flex-up' and reloan_flag = 0 and flagDisbursement = 1 then 'Flex-up' 
                  else new_loan_type end as LoanProduct,
    count (distinct digitalLoanAccountId) as BookedApps
FROM 
    `prj-prod-dataplatform.risk_credit_mis.loan_master_table` 
    where flagDisbursement = 1

group by 1,2
order by 1 desc,2
;
"""

BookedApps = client.query(sq).to_dataframe(progress_bar_type='tqdm')
print(f"The rows and columns in BookedApps are:\t {BookedApps.shape}")
BookedApps.head(10)

Job ID 59b39a02-de6a-4a0e-b558-98cf2c680528 successfully executed: 100%|██████████|
Downloading: 100%|██████████|
The rows and columns in BookedApps are:	 (192, 3)


,mm,LoanProduct,BookedApps
0,2025-01-01,ACL TSA,10
1,2025-01-01,Flex-up,50
2,2025-01-01,Quick,166
3,2025-01-01,Reloan,26
4,2025-01-01,SIL Competitor,38
5,2025-01-01,SIL Repeat,36
6,2025-01-01,SIL ZERO,41
7,2025-01-01,SIL-Instore,737
8,2024-12-01,ACL TSA,724
9,2024-12-01,Flex-up,870


In [9]:
# Get the total booked amount for a month

sq = """
# Booked Amount
SELECT 
DATE_TRUNC( disbursementDateTime,MONTH) as mm, 
    case when reloan_flag = 1 and loantype not like 'FLEXUP' and flagDisbursement = 1 then 'Reloan'
         when loantype = 'FLEXUP' and new_loan_type = 'Flex-up' and reloan_flag = 0 and flagDisbursement = 1 then 'Flex-up' 
                  else new_loan_type end as LoanProduct,
    sum (disbursedLoanAmount) as BookedAmt
FROM 
    `prj-prod-dataplatform.risk_credit_mis.loan_master_table` 
     where flagDisbursement = 1
group by 1,2
order by 1 desc,2
;
"""
BookedAmt = client.query(sq).to_dataframe(progress_bar_type='tqdm')
print(f"The rows and columns in BookedAmt are:\t {BookedAmt.shape}")
BookedAmt.head(10)

Job ID bb4e456f-12cd-428d-a616-622a9cefb434 successfully executed: 100%|██████████|
Downloading: 100%|██████████|
The rows and columns in BookedAmt are:	 (192, 3)


,mm,LoanProduct,BookedAmt
0,2025-01-01,ACL TSA,50000.00
1,2025-01-01,Flex-up,1637731.98
2,2025-01-01,Quick,2699900.00
3,2025-01-01,Reloan,524490.67
4,2025-01-01,SIL Competitor,481208.00
5,2025-01-01,SIL Repeat,503888.00
6,2025-01-01,SIL ZERO,586851.00
7,2025-01-01,SIL-Instore,7782890.00
8,2024-12-01,ACL TSA,3620000.00
9,2024-12-01,Flex-up,32119324.63


In [10]:
BookedAmt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 192 entries, 0 to 191
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   mm           192 non-null    datetime64[us]
 1   LoanProduct  192 non-null    object        
 2   BookedAmt    192 non-null    float64       
dtypes: datetime64[us](1), float64(1), object(1)
memory usage: 4.6+ KB


In [11]:
raw_data = StartedApps.merge(SubmittedApps,how='outer').merge(ApprovedApps,how='outer').merge(BookedApps,how='outer').merge(BookedAmt,how='outer')

In [12]:
raw_data.tail(10)

,mm,LoanProduct,StartedApps,SubmittedApps,ApprovedApps,BookedApps,BookedAmt
227,2024-12-01,SIL ZERO,3271,3238,1824,1814,2.868665e+07
228,2024-12-01,SIL-Instore,44351,43839,25870,25778,2.737510e+08
229,2025-01-01,ACL TSA,15,16,9,10,5.000000e+04
230,2025-01-01,Flex-up,34,34,34,50,1.637732e+06
231,2025-01-01,Quick,2571,2130,167,166,2.699900e+06
232,2025-01-01,Reloan,33,32,26,26,5.244907e+05
233,2025-01-01,SIL Competitor,71,55,40,38,4.812080e+05
234,2025-01-01,SIL Repeat,46,45,38,36,5.038880e+05
235,2025-01-01,SIL ZERO,90,91,44,41,5.868510e+05
236,2025-01-01,SIL-Instore,1418,1393,753,737,7.782890e+06


In [13]:
print(f'{data_year}-{data_month:02d}-01')

2024-12-01


In [14]:
raw_data_current_month = raw_data[(raw_data['mm']>='2022-08-01') &(raw_data['mm']<=f'{data_year}-{data_month:02d}-01')]

In [15]:
raw_data_current_month

,mm,LoanProduct,StartedApps,SubmittedApps,ApprovedApps,BookedApps,BookedAmt
56,2022-08-01,Flex,5558,4732,594,599,11985000.0
57,2022-08-01,Quick,43764,30660,1910,1926,33001000.0
58,2022-08-01,SIL-Instore,967,956,456,457,11379411.0
59,2022-08-01,SIL-Online(discontinued),1192,652,41,41,1197499.0
60,2022-09-01,Big Loan,13,12,<NA>,<NA>,NaN
...,...,...,...,...,...,...,...
224,2024-12-01,Reloan,635,607,584,584,11837905.1
225,2024-12-01,SIL Competitor,1697,1334,1169,1163,16467409.0
226,2024-12-01,SIL Repeat,1331,1313,1056,1054,14633772.0
227,2024-12-01,SIL ZERO,3271,3238,1824,1814,28686652.0


### Loan Account Tagging

In [16]:
sq="""
# Loan Tagging
SELECT
    loanAccountNumber,
    Case when reloan_flag = 1 and loantype not like 'FLEXUP' and flagDisbursement = 1 then 'Reloan'
         when loantype = 'FLEXUP' and new_loan_type = 'Flex-up' and reloan_flag = 0 and flagDisbursement = 1 then 'Flex-up' 
                  else new_loan_type end as LoanProduct,
    disbursementDateTime,
    (CASE WHEN new_loan_type = 'Flex-up' then
    LAG(new_loan_type) OVER (PARTITION BY customerId ORDER BY disbursementDateTime) END) AS OriginalLoanProduct
  FROM
    `prj-prod-dataplatform.risk_credit_mis.loan_master_table`
   where flagDisbursement=1
;
"""
loan_type = client.query(sq).to_dataframe(progress_bar_type='tqdm')
print(f"The rows and columns in loan_type are: {loan_type.shape}")
loan_type.tail(10)

Job ID e7946c44-d249-426b-9c63-325deb6cd79f successfully executed: 100%|██████████|
Downloading: 100%|██████████|
The rows and columns in loan_type are: (282417, 4)


,loanAccountNumber,LoanProduct,disbursementDateTime,OriginalLoanProduct
282407,60831000260019,SIL Competitor,2024-12-12 19:45:07,None
282408,60831109560017,SIL Competitor,2024-12-16 18:17:46,None
282409,60831111830011,SIL Competitor,2024-12-16 19:50:02,None
282410,60831280270012,SIL Competitor,2024-12-22 14:48:50,None
282411,60831423040011,SIL Competitor,2024-12-26 17:46:20,None
282412,60831424920019,SIL Competitor,2024-12-26 17:40:27,None
282413,60831426870018,SIL Competitor,2024-12-26 18:16:48,None
282414,60831453480014,SIL Competitor,2024-12-27 16:53:30,None
282415,60831533610018,SIL Competitor,2024-12-30 11:14:07,None
282416,60831550110017,SIL Competitor,2024-12-30 17:08:16,None


In [17]:
loan_type[loan_type['loanAccountNumber']=='60828136490016']

,loanAccountNumber,LoanProduct,disbursementDateTime,OriginalLoanProduct
149360,60828136490016,SIL-Instore,2024-09-01 12:25:12,None


In [18]:
Loan_Acs_current_month = loan_type[(loan_type['disbursementDateTime'].dt.month == data_month) & (loan_type['disbursementDateTime'].dt.year == data_year)]

In [19]:
Loan_Acs_current_month = Loan_Acs_current_month[['loanAccountNumber', 'LoanProduct','OriginalLoanProduct']]

#### VAS

In [20]:
sq = """
# Vas 
SELECT 
    extract(year from disbursementDateTime) as year , 
    extract(month from disbursementDateTime) as month,
    Case when reloan_flag = 1 and b.loantype not like 'FLEXUP' and flagDisbursement = 1 then 'Reloan'
         when b.loantype = 'FLEXUP' and new_loan_type = 'Flex-up' and reloan_flag = 0 and flagDisbursement = 1 then 'Flex-up' 
                  else new_loan_type end as new_loan_type, 
    count(distinct  b.loanAccountNumber ) AS TotalSold,
   
   count(distinct CASE WHEN vas_flag = 'true' THEN b.loanAccountNumber END) AS SoldWithVAS
     FROM `prj-prod-dataplatform.dl_loans_db_raw.tdbk_digital_loan_application` a join
`prj-prod-dataplatform.risk_credit_mis.loan_master_table` b on a.loanAccountNumber = b.loanAccountNumber
where flagDisbursement = 1
group by 1,2 ,3 
order by 1 desc,2 desc,3
;
"""

vas_df = client.query(sq).to_dataframe(progress_bar_type='tqdm')
print(f"The rows and columns in vas_df are:\t {vas_df.shape}")
vas_df.head(10)

Job ID 31991afd-ca98-43f7-9b56-b79ab48227a7 successfully executed: 100%|██████████|
Downloading: 100%|██████████|
The rows and columns in vas_df are:	 (192, 5)


,year,month,new_loan_type,TotalSold,SoldWithVAS
0,2025,1,ACL TSA,10,3
1,2025,1,Flex-up,50,22
2,2025,1,Quick,166,85
3,2025,1,Reloan,26,13
4,2025,1,SIL Competitor,38,26
5,2025,1,SIL Repeat,36,26
6,2025,1,SIL ZERO,41,29
7,2025,1,SIL-Instore,737,527
8,2024,12,ACL TSA,724,178
9,2024,12,Flex-up,870,386


In [21]:
vas_df_current_month = vas_df[(vas_df['year']==data_year)& (vas_df['month']==data_month)]

In [22]:
with pd.ExcelWriter(f'loan_accounts_tagging_{data_year}_{data_month:02d}_v1.xlsx') as writer:
    # Write each DataFrame to a specific sheet in the Excel file
    raw_data_current_month.to_excel(writer, sheet_name='raw_data', index=False)
    Loan_Acs_current_month.to_excel(writer, sheet_name='Loan Acs', index=False)
    vas_df_current_month.to_excel(writer, sheet_name='VAS', index=False)

In [23]:
print(f'loan_accounts_tagging_{data_year}_{data_month:02d}_v1.xlsx')

loan_accounts_tagging_2024_12_v1.xlsx
